In [0]:
%run "/Workspace/Formula1Project/includes/configuration"

In [0]:
%run "/Workspace/Formula1Project/includes/common_functions"

In [0]:
accounts_key = dbutils.secrets.get(scope="formala1-scope", key="account-key")
spark.conf.set("fs.azure.account.key.formala1.dfs.core.windows.net", accounts_key)

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
laptimes_schema = StructType([StructField("raceId", IntegerType(), False), 
                              StructField("driverId", IntegerType(), False), 
                              StructField("lap", IntegerType(), False), 
                              StructField("position", IntegerType(), False),
                              StructField("time", StringType(), False),
                              StructField("milliseconds", IntegerType(), False)])

In [0]:
laptimes_df = spark.read.csv(f"{raw_path}/{v_file_date}/lap_times",
                             schema=laptimes_schema)

In [0]:
laptimes_df.count()

In [0]:
from pyspark.sql.functions import current_timestamp,lit
laptimes_final_df = laptimes_df.withColumnRenamed("raceId", "race_id").withColumnRenamed("driverId", "driver_id").withColumn("ingestion_date",current_timestamp()).withColumn("file_date",lit(v_file_date))


In [0]:
merge_condition = "tgt.race_id = src.race_id and tgt.driver_id = src.driver_id and tgt.lap = src.lap"
merge_data(laptimes_final_df, processed_path, "f1_processed", "lap_times", "race_id", merge_condition)

In [0]:
#laptimes_final_df.write.mode("append").format("parquet").saveAsTable("f1_processed.laptimes")

In [0]:
%sql
select * from f1_processed.lap_times
order by file_date desc